In [ ]:
import numpy as np
from tqdm import tqdm
from pathlib import Path
from gtsam.symbol_shorthand import X
import matplotlib.pyplot as plt
import plotly.graph_objects as go

from lac.slam.gtsam_factor_graph import GtsamFactorGraph
from lac.slam.slam import SLAM
from lac.slam.visual_odometry import StereoVisualOdometry
from lac.slam.feature_tracker import FeatureTracker
from lac.utils.plotting import plot_poses, plot_surface, plot_3d_points
from lac.utils.visualization import image_grid
from lac.util import load_data, load_stereo_images, load_images

%load_ext autoreload
%autoreload 2

In [ ]:
# Load the data logs
data_path = "/home/shared/data_raw/LAC/runs/full_spiral_map1_preset0"
initial_pose, lander_pose, poses, imu_data, cam_config = load_data(data_path)
print(f"Loaded {len(poses)} poses")

In [ ]:
# images = load_images(data_path, cameras=["FrontLeft", "FrontRight"], start_frame=0, end_frame=10000)
left_imgs, right_imgs = load_stereo_images(data_path, start_frame=0, end_frame=10000)
images = {"FrontLeft": left_imgs, "FrontRight": right_imgs}

In [ ]:
# Load the ground truth map
map = np.load(
    "/home/shared/data_raw/LAC/heightmaps/competition/Moon_Map_01_preset_0.dat",
    allow_pickle=True,
)

In [ ]:
# Initialize modules
START_FRAME = 80

svo = StereoVisualOdometry(cam_config)
svo.initialize(poses[START_FRAME], images["FrontLeft"][START_FRAME], images["FrontRight"][START_FRAME])

tracker = FeatureTracker(cam_config)
tracker.initialize(poses[START_FRAME], images["FrontLeft"][START_FRAME], images["FrontRight"][START_FRAME])

graph = SLAM()
pose_key = 0
graph.add_pose(pose_key, poses[START_FRAME])
graph.add_vision_factors(pose_key, tracker.world_points, tracker.prev_pts, tracker.track_ids)

In [ ]:
IMG_RATE = 2
KEYFRAME_RATE = 10
GRAPH_UPDATE_RATE = 2
GRAPH_OPTIMIZE_RATE = 1000
END_FRAME = 2000

curr_pose = initial_pose
svo_poses = [initial_pose]
opt_first_key = pose_key

# Main loop over image frames
for step in tqdm(range(START_FRAME + IMG_RATE, END_FRAME + 1, IMG_RATE)):
    if step >= max(images["FrontLeft"].keys()):
        print("Reached end of images")
        break
    # Run VO for real-time pose tracking
    svo.track(images["FrontLeft"][step], images["FrontRight"][step])
    curr_pose = svo.get_pose()
    svo_poses.append(curr_pose)

    # Track features
    # tracker.track_keyframe(curr_pose, left_imgs[step], right_imgs[step])
    if step % KEYFRAME_RATE == 0:
        tracker.track_keyframe(curr_pose, images["FrontLeft"][step], images["FrontRight"][step])
    else:
        tracker.track(images["FrontLeft"][step])

    # Add to the graph
    if step % GRAPH_UPDATE_RATE == 0:
        pose_key += 1
        graph.add_pose(pose_key, curr_pose)
        graph.add_vision_factors(pose_key, tracker.world_points, tracker.prev_pts, tracker.track_ids)

    graph.add_odometry_factor(pose_key, svo.pose_delta)

    # # Optimize the graph
    # if step % GRAPH_OPTIMIZE_RATE == 0:
    #     print(f"Optimizing window {opt_first_key} to {pose_key}")
    #     window = list(range(opt_first_key, pose_key + 1))
    #     graph.optimize(window)
    #     opt_first_key = pose_key

In [ ]:
# graph.plot(start=0, end=1000, step=50)

In [ ]:
# Optimize the full graph
window = list(range(0, 900))
result, g, v = graph.optimize(window)

In [ ]:
fig = graph.plot(start=0, end=1000, step=50)
# fig = plot_surface(map, fig=fig, showscale=False)
fig = plot_poses(poses[:END_FRAME], fig=fig, no_axes=True, color="black", name="Ground truth")
fig = plot_poses(svo_poses, fig=fig, no_axes=True, color="orange", name="VO poses")
# fig = plot_3d_points(landmark_points_cropped, fig=fig, color="red", markersize=2, name="Landmarks")
fig.update_layout(height=900, width=1600, scene_aspectmode="data")
fig.show()

In [ ]:
graph.projection_factors[200][0].error(result)

In [ ]:
graph_poses = []
for pose in graph.poses.values():
    graph_poses.append(pose)

In [ ]:
fig = go.Figure()
# fig = plot_surface(map, fig=fig, showscale=False)
fig = plot_poses(poses[:END_FRAME], fig=fig, no_axes=True, color="black", name="Ground truth")
fig = plot_poses(svo_poses, fig=fig, no_axes=True, color="orange", name="VO poses")
fig = plot_poses(graph_poses, fig=fig, no_axes=True, color="green", name="Graph poses")
# fig = plot_3d_points(landmark_points_cropped, fig=fig, color="red", markersize=2, name="Landmarks")
fig.update_layout(height=900, width=1600, scene_aspectmode="data")
fig.show()

In [ ]:
graph.plot(start=0, end=100, step=10)

In [ ]:
fig.write_html("gtsam_slam.html")